# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [3]:
from asyncio import log
from cmath import cos, exp, sin, sqrt
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

import numpy as np

@dataclass
class Dual:
    value: float
    d: float
    
    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(self)}' and '{type(other)}'")    

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(float(other) - self.value, self.d)
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(self)}' and '{type(other)}'")


    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(self)}' and '{type(other)}'")


    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / (o_value ** 2))
            case Number():
                return Dual(self.value / float(other), self.d / float(other))
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(self)}' and '{type(other)}'")

    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value ** o_value, self.value ** o_value * (o_value * self.d / self.value + o_d * np.log(self.value)))
            case Number():
                return Dual(self.value ** float(other), self.d * float(other) * self.value ** (float(other) - 1))
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(self)}' and '{type(other)}'")

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
    __rsub__ = __sub__  # https://docs.python.org/3/reference/datamodel.html#object.__rsub__
    __rtruediv__ = __truediv__ # https://docs.python.org/3/reference/datamodel.html#object.__rtruediv__ 
    __rpow__ = __pow__  # https://docs.python.org/3/reference/datamodel.html#object.__rpow__


    # Задание 1.1
    # Унарный плюс
    def pos(x: Union[float, float]) -> Union[float, float]:
        match x:
            case Dual(value, d):
                return Dual(value, d)
            case float():
                return x
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(x)}'")  

    # Задание 1.1
    # Унарный минус
    def neg(x: Union[float, float]) -> Union[float, float]:
        match x:
            case Dual(value, d):
                return Dual(-value, -d)
            case float():
                return -x
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(x)}'")
                 

    # Задание 1.1
    # Функция деления
    def div(x: Union[float, float], y: Union[float, float]) -> Union[float, float]:
        match x:
            case Dual(value, d):
                return Dual(value / y, d / y)
            case float():
                return x / y
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(x)}' and '{type(y)}'")  


    # Задание 1.1
    # Возведение в степень
    def pow(x: Union[float, float], y: Union[float, float]) -> Union[float, float]:
        match x:
            case Dual(value, d):
                return Dual(value ** y, d * y * value ** (y - 1))
            case float():
                return x ** y 
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(x)}' and '{type(y)}'")    

    # Задание 1.2
    #  Функция sin(x)
    def sin(x: Union[float, float]) -> Union[float, float]:
        match x:
            case Dual(value, d):
                return Dual(np.sin(value), np.cos(value) * d)
            case float():
                return np.sin(x)
            case _: 
                raise ValueError(f"unsupported operand type(s) for +: '{type(x)}'")

    # Задание 1.2
    # Функция cos(x)
    def cos(x: Union[float, float]) -> Union[float, float]:
        match x:
            case Dual(value, d):
                return Dual(np.cos(value), -np.sin(value) * d)
            case float():
                return np.cos(x)
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(x)}'")
                

    # Задание 1.2
    # Функция exp(x)
    def exp(x: Union[float, float]) -> Union[float, float]:
        match x:
            case Dual(value, d):
                return Dual(np.exp(value), np.exp(value) * d)
            case float():
                return np.exp(x)
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(x)}'")


    # Задание 1.2
    # Функция log(x)
    def log(x: Union[float, float]) -> Union[float, float]:
        match x:
            case Dual(value, d):
                return Dual(np.log(value), d / value)
            case float():
                return np.log(x)
            case _:
                raise ValueError(f"unsupported operand type(s) for +: '{type(x)}'")

    # Задание 1.5
    # Реализуйте поддержку функций нескольких аргументов f(x, y, z)
    def f(x: Union[float, float], y: Union[float, float], z: Union[float, float]) -> Union[float, float]:
        return x * y + z
                

    
 
# Addition and Multiplication
def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

# Задание 1.5
# функций нескольких аргументов f(x, y, z)
def diff2(func: Callable[[float, float, float], float]) -> Callable[[float, float, float], float]:
    return lambda x, y, z: func(Dual(x, 2.0), Dual(y, 1.0), Dual(z, 3.0)).d
    
     

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [4]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 140*x + 72*x - 10

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)




212.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

### Ответь
Можно проверить корректость с помошю численного аппроксимиования значения в некоторых точках и через программого как вот это задачи которые мы решаем.

In [5]:
# Test for unary operators
def f(x: float) -> float:
    return 140*x + 27*x - 10

h_diff = diff(f)

# значение производной в точке x = 2
h_diff(2)


167.0

In [6]:
# Test for dvision operator
def p(x: float) -> float:
    return 5 / x**5 + 10

p_diff = diff(p)

# значение производной в точке x = 2
p_diff(2)

16.0

In [7]:
# Test for power operator
def g(x: float) -> float:
    return x**8 + 2*x**4 + 3*x + 1

j_diff = diff(g)

# значение производной в точке x = 2
j_diff(2)    

1091.0

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [8]:
# Test for exp() operator
def k(x: float) -> float:
    return np.exp(x) + 4*x**2 + 3*x + 1

k_diff = diff(k)

# значение производной в точке x = 2
k_diff(2)

26.389056098930652

In [9]:
# Test for cos() operator
def l(x: float) -> float:
    return np.cos(x) + 2*x**10 + 8*x + 1

l_diff = diff(l)

# значение производной в точке x = 2
l_diff(2)

10247.090702573174

In [10]:
# Test for sin() operator
def m(x: float) -> float:
    return np.sin(x) + 2*x**2 + 3*x + 1

m_diff = diff(m)

# значение производной в точке x = 2
m_diff(2)

10.583853163452858

In [11]:
# Test for log() operator
def n(x: float) -> float:
    return np.log(x) + 2*x**2 + 3*x + 1

n_diff = diff(n)

# значение производной в точке x = 2
n_diff(2)

11.5

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [12]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)



22.0

In [13]:
# Test for unary operators\
from scipy.misc import derivative

def h(x: float) -> float:
    return -2*x + 1 + 2*x*x

derivative(h, 2.)

6.0

In [14]:
# Тест на оператор деления
from scipy.misc import derivative

def i(x: float) -> float:
    return 1 / x**2 + 2

derivative(i, 2.)

-0.4444444444444444

In [15]:
# Тест на силового оператора
from scipy.misc import derivative

def j(x: float) -> float:
    return x**3 + 2*x**2 + 3*x + 1

derivative(j, 2.)

24.0

In [16]:
# Проверка оператора exp()
from scipy.misc import derivative

def k(x: float) -> float:
    return np.exp(x) + 2*x**2 + 3*x + 1

derivative(k, 2.)

19.68362754736431

In [17]:
# Проверка оператора cos()
from scipy.misc import derivative

def l(x: float) -> float:
    return np.cos(x) + 2*x**2 + 3*x + 1

derivative(l, 2.)

10.234852598765706

In [18]:
# Проверка оператора sin()
from scipy.misc import derivative

def m(x: float) -> float:
    return np.sin(x) + 2*x**2 + 3*x + 1

derivative(m, 2.)

10.649824511625987

In [19]:
# Проверка оператора log()
from scipy.misc import derivative

def n(s: float) -> float:
    return np.log(s) + 2*s**4 + 3*s + 1

derivative(n, 2.)

83.54930614433405

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [20]:
# генератор функций в виде строки

import random
import math
import random
from dataclasses import dataclass
from typing import Callable, List, Union

class FunctionGenerator:

    Maxlevel: int = 5

    def __init__(self, cmin: int = 0, cmax: int = 10) -> None:
        self.c_range = (cmin, cmax)
    def __gen_op(self, level: int) -> str:
        return random.choice([
            '({0} + {1})',
            '({0} - {1})',
            '({0} * {1})',
            '({0} / {1})',
            '({0} ** {1})',
        ]).format(self.generate(level - 1), self.generate(level - 1))

    def __gen_func(self, level: int) -> str:
        if level >= self.Maxlevel:
            return self.__gconst()
        else:
            return random.choice([
                'cos({0})',
                'sin({0})',
                'exp({0})',
                'log({0})',
            ]).format( self.generate(level - 1))    

    def __gen_leaf(self, level: int) -> str:
        return random.choice([
            'x',
            'x',
            'x',
            'x',
            str(random.randint(*self.c_range)),
        ])

    def __generate(self, level: int) -> str:
        return random.choice([
            self.__gen_func,
            self.__gen_op,
            self.__gen_leaf,
        ])(level - 1)   

    def generate(self, level: int = 5) -> str:
        self.Maxlevel = level
        return self.__generate(level)

if __name__ == '__main__':
    fg = FunctionGenerator()
    for i in range(10):
        print(fg.generate(5))



log(sin(x))
exp(x)
6
4
x
(sin(x) + ((4 / x) ** 7))
0
(x / sin((x / x)))
x
x


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x*y + z -5*y + 2*x


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [21]:
# Test for matmul() operator
def f(x: float, y: float, z: float) -> float:
    return x*y + z -5*y + 2*x

f_diff = diff2(f)

# значение производной в точке x = 10 y = 20 z = 10
f_diff(10 , 20, 10) 

52.0